In [ ]:
import torch
import torch.nn as nn
import math

In [ ]:
class LayerNormalization(nn.Module):
    """
         Normalización de capas para ajustar los valores de entrada y estabilizar el entrenamiento.
         Esto con el fin de no tener valores disparejos y obtener una media para todos (media ajustada).
    """
    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features)) # alpha is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features)) # bias is a learnable parameter

    def forward(self, x):
        '''
             Ajustar los valores usando la media, desviacion, alpha y bias.
        '''
        mean = x.mean(dim = -1, keepdim = True) # Media de la ultima dimension
        std = x.std(dim = -1, keepdim = True)  #Desviasión estandar
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

In [ ]:
class FeedForwardBlock(nn.Module):
    """
        Aplicar dos transformaciones lineales con una activación no lineal (ReLU intermedia) y un dropout entre ellas
    """
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # capa
        self.dropout = nn.Dropout(dropout) # Dropout para regularizacion
        self.linear_2 = nn.Linear(d_ff, d_model) # capa

    def forward(self, x):
        '''
            Se aplican todos los elementos. En este caso, la transformación lineal, activación, dropout y luego la segunda transformación
        '''
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

In [ ]:
class InputEmbeddings(nn.Module):
    """
        Convierte los índices de vocabulario en vectores densos (embeddings)
        de tamaño d_model.
    """
    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.d_model = d_model # Dimension
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        '''
             Multiplicación por la raíz cuadrada de d_model para los vectores densos.
        '''
        return self.embedding(x) * math.sqrt(self.d_model)
    

In [ ]:
class PositionalEncoding(nn.Module):
    """
         Ayuda para poder obtener el orden de las palabras, agregando información posicional a los embeddings de entrada  
    """
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(seq_len, d_model) # Generar posiciones
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # Calcular frecuencia de cada dimensión
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # aplicar seno y coseno
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term) 
        pe = pe.unsqueeze(0) 
        self.register_buffer('pe', pe)

    def forward(self, x):
        '''
             Agregar la posicion a los embeddings
        '''
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)

In [ ]:
class ResidualConnection(nn.Module):
    """
        Aplica normalización de capas y dropout.
    """    
        def __init__(self, features: int, dropout: float) -> None:
            super().__init__()
            self.dropout = nn.Dropout(dropout)
            self.norm = LayerNormalization(features)
    
        def forward(self, x, sublayer):
            '''
                Aplicar el dropout y normalización.
            '''
            return x + self.dropout(sublayer(self.norm(x)))


In [ ]:
class MultiHeadAttentionBlock(nn.Module):
    """
        Se calcula la ponderación entre las entradas, esto ayuda a determinar
        la importancia en cada token, es decir, que tenga sentido o no en la
        sentencia.
    """

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model 
        self.h = h # Cabezas
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h 
        # Proyecciones Q, K y V
        self.w_q = nn.Linear(d_model, d_model, bias=False)
        self.w_k = nn.Linear(d_model, d_model, bias=False) 
        self.w_v = nn.Linear(d_model, d_model, bias=False) 
        self.w_o = nn.Linear(d_model, d_model, bias=False) 
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        '''
            Puntaje de la atencion
        '''
        d_k = query.shape[-1]
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) 
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        '''
            Aplicar proyecciones
        '''
        query = self.w_q(q) 
        key = self.w_k(k) 
        value = self.w_v(v) 
        # Dividir las matrices (K) en varias cabezas
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        return self.w_o(x)

In [ ]:
class EncoderBlock(nn.Module):
    """ 
         Analiza la relevansia de una palabra en base al contexto de la misma.
         Es decir, mediante el Encoder, recibe la secuencia y analiza cada una
         de las palabras (tokens) obteniendo el valor de cada palabra en el texto.
         Si es mas importante por no estar tantas veces, si aparece muchas veces y asi.
    """

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        '''
            Aplicar conexiones residuales para permitir que la red aprenda 
            identidades sin perder la capacidad de transformar las entradas.
        '''
        # Aplicar al attention_block la primera conexión residual 
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        # Aplicar al feed-forward la 2da conexion residual
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x
    

In [ ]:
class Encoder(nn.Module):
    """
     Procesa las entradas través de varias capas de atención y transformación,
     obteniendo el contexto de los demas tokens para 1 solo. Es decir, obtener
     un panorama general. Este se conforma de varios EncoderBlock para realizar
     la tarea. Es decir, obtiene el texto completo y con el encoderBlock analiza
     su relevancia
    """

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        '''
            Aplicar la transformación y la máscara para cada codificador.
        '''
        for layer in self.layers:
            x = layer(x, mask)
        # Normalizar
        return self.norm(x)


In [ ]:
class DecoderBlock(nn.Module):
    """
        Obtiene la secuencia de valores, para poder transcribirlos uno a uno
        y o
    """

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        # Conexiones residuales
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x
    

In [ ]:
class Decoder(nn.Module):
    """
        Posee capas de DecoderBlock, para poder transformar las secuencias
        objetivo a partir de la inforamcion del codificador.
        Esto con el fin de poder obtener la data y normalizarla luego de
        para que sea exacta y no tenga ruido. En este caso, obtiene el valor
        que se le da al texto, para poder traducirlo.
    """

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        # Normalizar
        return self.norm(x)


In [ ]:
class ProjectionLayer(nn.Module):
    """
        Convierte las salidas de dimensión d_model a una distribución de 
        probabilidad sobre el vocabulario de salida, que es lo que 
        finalmente predice el modelo.
    """
    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
        return self.proj(x)
    

In [ ]:
class Transformer(nn.Module):
    """
        Conector de las capras codificador, decodificador y proyección. 
        Este ayuda a que funcione el proceso en si de todos las clases previas
        para que se haga el proceso de codificacion, embedding, decodicacion y proyeccion
        con el fin de hallar similitudes en palabras, o poder categorizarlas.
    """

    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # Embebemr la secuencia de entrada y codificación posicional.
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)
    
    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        # Embeber la secuencia objetivo y  codificación posicional.
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)
    
    def project(self, x):
        # (batch, seq_len, vocab_size). Probabiliades sobre vocabulario.
        return self.projection_layer(x)
    

In [ ]:
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
    '''
        Metodo de inicializacion para la construccion del transformador. En este caso es para inicializar el proceso de la
        red y contruirla en base a las entradas de texto que se vayan pasando. Esto con el fin de hallar traducciones, relaciones entre
        palabras o lo que se desee realizar con la red.
    '''
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)
    
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)
    
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))
    
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)
    
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    
    return transformer